In [ ]:
import json
import sys
import MetaTrader5 as mt5
import pandas as pd
from datetime import datetime
import pytz
import time
from joblib import load
import logging
from tsfresh import extract_features
from tsfresh.utilities.dataframe_functions import roll_time_series, impute
import talib
import os
import threading

logging.basicConfig(filename='trading_GBPUSD_USDCAD_Buy_Sell_D1.log', level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Initialize connection to MetaTrader 5
def init_mt5_connection(login, password, server):
    if not mt5.initialize(login=login, password=password, server=server):
        logging.error(f"initialize() failed, error code = {mt5.last_error()}")
        sys.exit()
    logging.info("Connected to MetaTrader 5")
    print("Connected to MetaTrader 5")

# Fetch the latest candle
def fetch_latest_candle(symbol, timeframe):
    latest_candles = mt5.copy_rates_from_pos(symbol, timeframe, 0, 1)
    if latest_candles is None or len(latest_candles) == 0:
        logging.error(f"Failed to fetch latest candle for {symbol}")
        return None
    return latest_candles[0]


# Fetch historical data
def fetch_historical_data(symbol, timeframe, start_date, end_date):
    logging.info(f"Fetching historical data for {symbol}")
    print(f"Fetching historical data for {symbol}")
    data = mt5.copy_rates_range(symbol, timeframe, start_date, end_date)
    ohlc_data = pd.DataFrame(data)
    ohlc_data['time'] = pd.to_datetime(ohlc_data['time'], unit='s')
    return ohlc_data[['time', 'open', 'high', 'low', 'close']]

# Add rolling features
def add_rolling_features(df, window):
    logging.info("Adding rolling features")
    df['rolling_mean_open'] = df['open'].rolling(window=window).mean()
    df['rolling_std_open'] = df['open'].rolling(window=window).std()
    df['rolling_mean_close'] = df['close'].rolling(window=window).mean()
    df['rolling_std_close'] = df['close'].rolling(window=window).std()
    df['rolling_mean_high'] = df['high'].rolling(window=window).mean()
    df['rolling_std_high'] = df['high'].rolling(window=window).std()
    df['rolling_mean_low'] = df['low'].rolling(window=window).mean()
    df['rolling_std_low'] = df['low'].rolling(window=window).std()
    return df

# Add lag features
def add_lag_features(df, lags):
    logging.info("Adding lag features")
    for lag in lags:
        df[f'open_lag_{lag}'] = df['open'].shift(lag)
        df[f'close_lag_{lag}'] = df['close'].shift(lag)
        df[f'high_lag_{lag}'] = df['high'].shift(lag)
        df[f'low_lag_{lag}'] = df['low'].shift(lag)
    return df

# Calculate indicators
def calculate_indicators(df):
    logging.info("Calculating indicators")
    for period in [15, 23, 42, 145]:
        df[f'WILLR_{period}'] = talib.WILLR(df['high'], df['low'], df['close'], timeperiod=period)
    return df

# Process data for feature extraction
def process_data_for_features(df, symbol, signals, selected_features):
    logging.info(f"Processing data for feature extraction for {symbol}")
    X_combined = pd.DataFrame(index=df['time'])
    
    for signal in signals:
        df_melted = df[['time', signal]].copy()
        df_melted["Symbols"] = symbol
        df_rolled = roll_time_series(df_melted, column_id="Symbols", column_sort="time",
                                     max_timeshift=20, min_timeshift=5)
        X = extract_features(df_rolled.drop("Symbols", axis=1), column_id="id", column_sort="time", 
                             column_value=signal, impute_function=impute, show_warnings=False)
        X = X.set_index(X.index.map(lambda x: x[1]), drop=True)
        
        # Filter features
        selected_features_X = [feature for feature in selected_features if feature in X.columns]
        X_filtered = X[selected_features_X]
        
        X_combined = X_combined.merge(X_filtered, left_index=True, right_index=True, how='left')
        X_combined = X_combined.dropna()
    return X_combined

# Combine buy/sell data
def fetch_and_process_data(symbol, timeframe, signals, selected_features, start_date, end_date):
    logging.info(f"Fetching and processing data for {symbol}")
    print(f"Fetching and processing data for {symbol}")
    
    df = fetch_historical_data(symbol, timeframe, start_date, end_date)
    df = calculate_indicators(df)
    df = add_rolling_features(df, window=5)
    df = add_lag_features(df, lags=[1, 2, 3, 4, 5])
    
    df = df.dropna().reset_index(drop=True)
    
    features = process_data_for_features(df, symbol, signals, selected_features)
    df = df.set_index('time')
    combined_df = df.merge(features, left_index=True, right_index=True, how='left')
    
    # Handle missing features by filling with zeros
    missing_features = [feat for feat in selected_features if feat not in combined_df.columns]
    for feat in missing_features:
        combined_df[feat] = 0
    
    combined_df = combined_df[selected_features]
    
    combined_df = combined_df.dropna()
    
    return combined_df


# Calculate SL and TP based on entry price and specified percentages
def calculate_prices(entry_price, risk_reward_ratio, mean_candle_size):
    logging.info("Calculating SL and TP prices")
    risk_part, reward_part = map(int, risk_reward_ratio.split(':'))
    risk_amount = mean_candle_size * risk_part
    reward_amount = mean_candle_size * reward_part
    sl_price = entry_price - risk_amount
    tp_price = entry_price + reward_amount
    return sl_price, tp_price

# Place order
def place_order(symbol, volume, sl_price, tp_price, trade_type):
    logging.info(f"Placing {trade_type} order for {symbol}")
    
    # Check if the market is open for the symbol
    symbol_info = mt5.symbol_info(symbol)
    if symbol_info is None:
        logging.error(f"Failed to get symbol info for {symbol}.")
        return
    
    if not symbol_info.visible or symbol_info.trade_mode == mt5.SYMBOL_TRADE_MODE_DISABLED:
        logging.error(f"Market for {symbol} is currently closed or not available for trading.")
        print(f"Market for {symbol} is closed. Cannot place order.")
        return

    # Proceed with order placement
    price = mt5.symbol_info_tick(symbol).ask if trade_type == "Buy" else mt5.symbol_info_tick(symbol).bid
    order_type = mt5.ORDER_TYPE_BUY if trade_type == "Buy" else mt5.ORDER_TYPE_SELL
    
    request = {
        "action": mt5.TRADE_ACTION_DEAL,
        "symbol": symbol,
        "volume": volume,
        "type": order_type,
        "price": price,
        "sl": sl_price,
        "tp": tp_price,
        "deviation": 10,
        "magic": 0,
        "comment": f"Python script {trade_type} order",
        "type_time": mt5.ORDER_TIME_GTC,
        "type_filling": mt5.ORDER_FILLING_IOC,
    }
    
    result = mt5.order_send(request)
    
    if result and result.retcode == mt5.TRADE_RETCODE_DONE:
        logging.info(f"{trade_type} order placed successfully for {symbol}.")
        print(f"{trade_type} order placed successfully for {symbol}.")
    else:
        logging.error(f"Order placement failed for {symbol}: {result.retcode if result else 'No response'} - {mt5.last_error()}")
        print(f"Order placement failed for {symbol}: {result.retcode if result else 'No response'} - {mt5.last_error()}")

# Function to apply the custom threshold
def apply_custom_threshold(probas, custom_threshold):
    return (probas >= custom_threshold).astype(int)

# Function to track predictions for both buy and sell to avoid simultaneous trades
def update_trade_decision(symbol, trade_type, pred, trade_decisions):
    if trade_type == "Buy":
        trade_decisions[symbol]['buy'] = pred
    elif trade_type == "Sell":
        trade_decisions[symbol]['sell'] = pred

    # If both buy and sell are 1, no trade should be taken
    if trade_decisions[symbol]['buy'] == 1 and trade_decisions[symbol]['sell'] == 1:
        logging.info(f"Both Buy and Sell predictions are 1 for {symbol}. No trade will be taken.")
        print(f"Both Buy and Sell predictions are 1 for {symbol}. No trade will be taken.")
        return False
    return True

# Predict and trade for Buy or Sell with check for simultaneous signals
def predict_and_trade(scaler, model, symbol, volume, timeframe, risk_reward_ratio, mean_candle_size, start_date, end_date, features, trade_type, trade_decisions, threshold=None):
    logging.info(f"Starting prediction and trading process for {symbol} - {trade_type}")
    
    signals = ['WILLR_15', 'WILLR_42']  # Signals used for both buy and sell

    # Fetch and process data
    df_processed = fetch_and_process_data(symbol, timeframe, signals, features, start_date, end_date)

    # Shift the data
    original_index = df_processed.index
    df_processed = df_processed.shift(periods=1, axis=0).dropna()

    # Align the features with what the scaler expects
    df_processed = df_processed[features]

    # Scale the data
    scaled_data = scaler.transform(df_processed)
    
    # Predict probabilities and apply the provided threshold or default to 0.5
    probas = model.predict_proba(scaled_data)[:, 1]
    custom_threshold = threshold if threshold is not None else 0.5  # Use provided threshold or default to 0.5
    y_pred = apply_custom_threshold(probas, custom_threshold)
    
    # Create a DataFrame to hold prediction data
    df_pred = pd.DataFrame(index=df_processed.index)
    df_pred['prediction'] = y_pred
    
    # Save predictions to CSV
    save_predictions_to_csv(df_pred, symbol, trade_type)
    
    # Update trade decisions for the symbol
    pred_decision = 1 if len(y_pred) > 1 and y_pred[-1] == 1 else 0
    should_trade = update_trade_decision(symbol, trade_type, pred_decision, trade_decisions)
    
    if not should_trade:
        logging.info(f"Skipping trade for {symbol} due to both Buy and Sell signals being active.")
        return

    if pred_decision == 1:  # If the prediction is a signal
        entry_price = mt5.symbol_info_tick(symbol).ask if trade_type == "Buy" else mt5.symbol_info_tick(symbol).bid
        sl_price, tp_price = calculate_prices(entry_price, risk_reward_ratio, mean_candle_size)
        place_order(symbol, volume, sl_price, tp_price, trade_type)
    else:
        logging.info(f"No {trade_type} signal generated for {symbol}. Doing nothing.")
        print(f"No {trade_type} signal generated for {symbol}. Doing nothing.")

def save_predictions_to_csv(df_pred, symbol, trade_type):
    file_path = f'predict_{symbol}_D1_3112_{trade_type}.csv'
    df_pred.to_csv(file_path, index=True)

# Function to process each pair for both buy and sell models with checks for missing models
def process_pair(config, utc_from, utc_to, trade_decisions):
    logging.info(f"Processing pair: {config['symbol']} for Buy and Sell")

    # Initialize variables for model/scaler availability
    buyscaler, buymodel, sellscaler, sellmodel = None, None, None, None

    # Try to load buy model and scaler
    try:
        buyscaler = load(config["buy_scaler_path"])
        buymodel = load(config["buy_model_path"])
        
        # Load buy features
        with open(config["buy_features_path"], 'r') as f:
            buy_features = json.load(f)
    except Exception as e:
        logging.warning(f"Buy model or scaler missing for {config['symbol']}: {e}")
        print(f"Buy model or scaler missing for {config['symbol']}: {e}")

    # Try to load sell model and scaler
    try:
        if "sell_scaler_path" in config and "sell_model_path" in config:
            sellscaler = load(config["sell_scaler_path"])
            sellmodel = load(config["sell_model_path"])
            
            # Load sell features
            with open(config["sell_features_path"], 'r') as f:
                sell_features = json.load(f)
        else:
            logging.warning(f"Sell model or scaler missing for {config['symbol']}")
            print(f"Sell model or scaler missing for {config['symbol']}")
    except Exception as e:
        logging.warning(f"Error loading sell model for {config['symbol']}: {e}")
        print(f"Error loading sell model for {config['symbol']}: {e}")

    last_candle = fetch_latest_candle(config["symbol"], config["timeframe"])
    if last_candle is None:
        logging.error(f"Failed to fetch initial candle data for {config['symbol']}.")
        print(f"Failed to fetch initial candle data for {config['symbol']}.")
        return

    try:
        while True:
            # Get the current UTC time
            current_time = datetime.now(pytz.utc)

            # Check if it's 00:20 UTC
            if current_time.hour == 0 and current_time.minute == 25:
                current_candle = fetch_latest_candle(config["symbol"], config["timeframe"])
                if current_candle and current_candle['open'] == last_candle['open']:
                    # Predict and trade for Buy if model is available
                    if buyscaler is not None and buymodel is not None:
                        predict_and_trade(
                            buyscaler, buymodel, config["symbol"], config["volume"], config["timeframe"],
                            config["buy_risk_reward_ratio"], config["mean_candle_size"], utc_from, utc_to, buy_features, "Buy",
                            trade_decisions, threshold=config.get("buy_threshold", None)
                        )
                    
                    # Predict and trade for Sell if model is available
                    if sellscaler is not None and sellmodel is not None:
                        predict_and_trade(
                            sellscaler, sellmodel, config["symbol"], config["volume"], config["timeframe"],
                            config["sell_risk_reward_ratio"], config["mean_candle_size"], utc_from, utc_to, sell_features, "Sell",
                            trade_decisions, threshold=config.get("sell_threshold", None)
                        )

                    last_candle = current_candle
                logging.info(f"Finished trading for {config['symbol']} at 00:20 UTC")

            else:
                # Sleep for 60 seconds and check again if it's 00:20 UTC
                time.sleep(60)
    except KeyboardInterrupt:
        logging.info(f"Script terminated by user for {config['symbol']}.")
        print(f"Script terminated by user for {config['symbol']}.")
    finally:
        mt5.shutdown()
        logging.info(f"MetaTrader 5 connection closed for {config['symbol']}.")
        print(f"MetaTrader 5 connection closed for {config['symbol']}.")


# Main function
def main():
    # Sample configuration, replace with your actual configuration details
    config = {'login': 51708234, 'password': '4bM&wuVJcBTnjV', 'server': 'ICMarketsEU-Demo'}

    init_mt5_connection(config['login'], config['password'], config['server'])
    
    # Configuration for GBPUSD (Only Buy model provided, no threshold specified)
    gbpusd_config = {
        "symbol": "GBPUSD",
        "timeframe": mt5.TIMEFRAME_D1,
        "volume": 0.1,
        "mean_candle_size": 0.013,
        "buy_risk_reward_ratio": "1:1",
        "sell_risk_reward_ratio": "1:1",
        "buy_scaler_path": 'GBPUSD_D1_3112buy/scaler.joblib',
        "buy_model_path": 'GBPUSD_D1_3112buy/model.joblib',
        "sell_scaler_path": 'GBPUSD_D1_3112sell/scaler.joblib',
        "sell_model_path": 'GBPUSD_D1_3112sell/model.joblib',
        "buy_features_path": 'GBPUSD_D1_3112buy/feature_names.json',
        "sell_features_path": 'GBPUSD_D1_3112sell/feature_names.json'
    }

    # Configuration for USDCAD (Both Buy and Sell models provided, no thresholds specified)
    usdcad_config = {
        "symbol": "USDCAD",
        "timeframe": mt5.TIMEFRAME_D1,
        "volume": 0.1,
        "mean_candle_size": 0.0088,
        "buy_risk_reward_ratio": "2:3",
        "sell_risk_reward_ratio": "2:3",
        "buy_scaler_path": 'USDCAD_D1_3112_BuyNEW/scaler.joblib',
        "buy_model_path": 'USDCAD_D1_3112_BuyNEW/model.joblib',
        "sell_scaler_path": 'USDCAD_D1_3112_Sell/scaler.joblib',
        "sell_model_path": 'USDCAD_D1_3112_Sell/model.joblib',
        "buy_features_path": 'USDCAD_D1_3112_BuyNEW/feature_names.json',
        "sell_features_path": 'USDCAD_D1_3112_Sell/feature_names.json'
    }
    eurusd_config = {
        "symbol": "EURUSD",
        "timeframe": mt5.TIMEFRAME_D1,
        "volume": 0.1,
        "mean_candle_size": 0.0105,
        "buy_risk_reward_ratio": "2:3",
        "buy_scaler_path": 'EURUSD_D1_3112buy/scaler.joblib',
        "buy_model_path": 'EURUSD_D1_3112buy/model.joblib',
        "buy_features_path": 'EURUSD_D1_3112buy/feature_names.json',
    }

    # Load the configurations
    configs = [gbpusd_config, usdcad_config,eurusd_config]

    utc_from = datetime(2023, 5, 1, tzinfo=pytz.utc)
    utc_to = datetime.now(pytz.utc)

    # Create a dictionary to store trade decisions for each symbol
    trade_decisions = {config['symbol']: {'buy': 0, 'sell': 0} for config in configs}

    threads = []
    for config in configs:
        thread = threading.Thread(target=process_pair, args=(config, utc_from, utc_to, trade_decisions))
        thread.start()
        threads.append(thread)

    for thread in threads:
        thread.join()

if __name__ == "__main__":
    main()


In [ ]:
import json
import sys
import MetaTrader5 as mt5
import pandas as pd
from datetime import datetime
import pytz
import time
from joblib import load
import logging
from tsfresh import extract_features
from tsfresh.utilities.dataframe_functions import roll_time_series, impute
import talib
import threading

logging.basicConfig(filename='trading_GBPUSD_USDCAD_Buy_Sell_D1.log', level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Initialize connection to MetaTrader 5
def init_mt5_connection(login, password, server):
    if not mt5.initialize(login=login, password=password, server=server):
        logging.error(f"initialize() failed, error code = {mt5.last_error()}")
        sys.exit()
    logging.info("Connected to MetaTrader 5")
    print("Connected to MetaTrader 5")

# Fetch historical data
def fetch_historical_data(symbol, timeframe, start_date, end_date):
    logging.info(f"Fetching historical data for {symbol}")
    data = mt5.copy_rates_range(symbol, timeframe, start_date, end_date)
    ohlc_data = pd.DataFrame(data)
    ohlc_data['time'] = pd.to_datetime(ohlc_data['time'], unit='s')
    return ohlc_data[['time', 'open', 'high', 'low', 'close']]

# Add rolling features
def add_rolling_features(df, window):
    logging.info("Adding rolling features")
    df['rolling_mean_open'] = df['open'].rolling(window=window).mean()
    df['rolling_std_open'] = df['open'].rolling(window=window).std()
    df['rolling_mean_close'] = df['close'].rolling(window=window).mean()
    df['rolling_std_close'] = df['close'].rolling(window=window).std()
    df['rolling_mean_high'] = df['high'].rolling(window=window).mean()
    df['rolling_std_high'] = df['high'].rolling(window=window).std()
    df['rolling_mean_low'] = df['low'].rolling(window=window).mean()
    df['rolling_std_low'] = df['low'].rolling(window=window).std()
    return df

# Add lag features
def add_lag_features(df, lags):
    logging.info("Adding lag features")
    for lag in lags:
        df[f'open_lag_{lag}'] = df['open'].shift(lag)
        df[f'close_lag_{lag}'] = df['close'].shift(lag)
        df[f'high_lag_{lag}'] = df['high'].shift(lag)
        df[f'low_lag_{lag}'] = df['low'].shift(lag)
    return df

# Calculate indicators
def calculate_indicators(df):
    logging.info("Calculating indicators")
    for period in [15, 23, 42, 145]:
        df[f'WILLR_{period}'] = talib.WILLR(df['high'], df['low'], df['close'], timeperiod=period)
    return df

# Align features to match the trained scaler/model
def align_features(df, expected_features):
    # Add missing features as columns with default value 0
    missing_features = [feature for feature in expected_features if feature not in df.columns]
    for feature in missing_features:
        df[feature] = 0
    
    # Remove unexpected features
    df = df[expected_features]
    
    return df


# Process data for feature extraction
def process_data_for_features(df, symbol, signals, selected_features):
    logging.info(f"Processing data for feature extraction for {symbol}")
    X_combined = pd.DataFrame(index=df['time'])
    
    for signal in signals:
        df_melted = df[['time', signal]].copy()
        df_melted["Symbols"] = symbol
        df_rolled = roll_time_series(df_melted, column_id="Symbols", column_sort="time", max_timeshift=20, min_timeshift=5)
        X = extract_features(df_rolled.drop("Symbols", axis=1), column_id="id", column_sort="time", column_value=signal, impute_function=impute, show_warnings=False)
        X = X.set_index(X.index.map(lambda x: x[1]), drop=True)
        
        selected_features_X = [feature for feature in selected_features if feature in X.columns]
        X_filtered = X[selected_features_X]
        
        X_combined = X_combined.merge(X_filtered, left_index=True, right_index=True, how='left')
        X_combined = X_combined.dropna()
    return X_combined

# Combine buy/sell data
def fetch_and_process_data(symbol, timeframe, signals, selected_features, start_date, end_date):
    logging.info(f"Fetching and processing data for {symbol}")
    
    df = fetch_historical_data(symbol, timeframe, start_date, end_date)
    df = calculate_indicators(df)
    df = add_rolling_features(df, window=5)
    df = add_lag_features(df, lags=[1, 2, 3, 4, 5])
    
    df = df.dropna().reset_index(drop=True)
    
    features = process_data_for_features(df, symbol, signals, selected_features)
    df = df.set_index('time')
    combined_df = df.merge(features, left_index=True, right_index=True, how='left')
    
    combined_df = align_features(combined_df, selected_features)  # Align features here
    combined_df = combined_df.dropna()
    
    return combined_df

# Calculate SL and TP based on entry price and specified percentages
def calculate_prices(entry_price, risk_reward_ratio, mean_candle_size):
    logging.info("Calculating SL and TP prices")
    risk_part, reward_part = map(int, risk_reward_ratio.split(':'))
    risk_amount = mean_candle_size * risk_part
    reward_amount = mean_candle_size * reward_part
    sl_price = entry_price - risk_amount
    tp_price = entry_price + reward_amount
    return sl_price, tp_price

# Place order
def place_order(symbol, volume, sl_price, tp_price, trade_type):
    logging.info(f"Placing {trade_type} order for {symbol}")
    
    symbol_info = mt5.symbol_info(symbol)
    if symbol_info is None or not symbol_info.visible:
        logging.error(f"Failed to get symbol info or symbol is not visible for {symbol}.")
        mt5.symbol_select(symbol, True)
        return
    
    if symbol_info.trade_mode == mt5.SYMBOL_TRADE_MODE_DISABLED:
        logging.error(f"Market for {symbol} is currently closed or not available for trading.")
        return
    
    # Check volume limits
    if volume < symbol_info.volume_min or volume > symbol_info.volume_max:
        logging.error(f"Volume {volume} is out of the allowed range for {symbol}. Min: {symbol_info.volume_min}, Max: {symbol_info.volume_max}")
        return

    price = mt5.symbol_info_tick(symbol).ask if trade_type == "Buy" else mt5.symbol_info_tick(symbol).bid
    order_type = mt5.ORDER_TYPE_BUY if trade_type == "Buy" else mt5.ORDER_TYPE_SELL
    
    request = {
        "action": mt5.TRADE_ACTION_DEAL,
        "symbol": symbol,
        "volume": volume,
        "type": order_type,
        "price": price,
        "sl": sl_price,
        "tp": tp_price,
        "deviation": 20,  # Try increasing the deviation
        "magic": 0,
        "comment": f"Python script {trade_type} order",
        "type_time": mt5.ORDER_TIME_GTC,
        "type_filling": mt5.ORDER_FILLING_IOC,
    }
    
    result = mt5.order_send(request)
    
    if result and result.retcode == mt5.TRADE_RETCODE_DONE:
        logging.info(f"{trade_type} order placed successfully for {symbol}.")
        print(f"{trade_type} order placed successfully for {symbol}.")
    else:
        logging.error(f"Order placement failed for {symbol}: {result.retcode if result else 'No response'} - {mt5.last_error()}")
        print(f"Order placement failed for {symbol}: {result.retcode if result else 'No response'} - {mt5.last_error()}")

# Function to apply the custom threshold
def apply_custom_threshold(probas, custom_threshold):
    return (probas >= custom_threshold).astype(int)

# Predict and trade for Buy or Sell with check for simultaneous signals
def predict_and_trade(scaler, model, symbol, volume, timeframe, risk_reward_ratio, mean_candle_size, start_date, end_date, features, trade_type, trade_decisions, threshold=None):
    logging.info(f"Starting prediction and trading process for {symbol} - {trade_type}")
    
    signals = ['WILLR_15', 'WILLR_42']  # Signals used for both buy and sell

    # Fetch and process data
    df_processed = fetch_and_process_data(symbol, timeframe, signals, features, start_date, end_date)

    # Shift the data
    df_processed = df_processed.shift(periods=1, axis=0).dropna()

    # Align the features with what the scaler expects
    df_processed = align_features(df_processed, features)

    # Scale the data
    scaled_data = scaler.transform(df_processed)
    
    # Predict probabilities and apply the provided threshold or default to 0.5
    probas = model.predict_proba(scaled_data)[:, 1]
    custom_threshold = threshold if threshold is not None else 0.5  # Use provided threshold or default to 0.5
    y_pred = apply_custom_threshold(probas, custom_threshold)
    
    # Create a DataFrame to hold prediction data
    df_pred = pd.DataFrame(index=df_processed.index)
    df_pred['prediction'] = y_pred
    
    # Save predictions to CSV
    save_predictions_to_csv(df_pred, symbol, trade_type)
    
    # Update trade decisions for the symbol
    pred_decision = 1 if len(y_pred) > 1 and y_pred[-1] == 1 else 0
    should_trade = update_trade_decision(symbol, trade_type, pred_decision, trade_decisions)
    
    if not should_trade:
        logging.info(f"Skipping trade for {symbol} due to both Buy and Sell signals being active.")
        return

    if pred_decision == 1:  # If the prediction is a signal
        entry_price = mt5.symbol_info_tick(symbol).ask if trade_type == "Buy" else mt5.symbol_info_tick(symbol).bid
        sl_price, tp_price = calculate_prices(entry_price, risk_reward_ratio, mean_candle_size)
        place_order(symbol, volume, sl_price, tp_price, trade_type)
    else:
        logging.info(f"No {trade_type} signal generated for {symbol}. Doing nothing.")

def save_predictions_to_csv(df_pred, symbol, trade_type):
    file_path = f'predict_{symbol}_D1_3112_{trade_type}.csv'
    df_pred.to_csv(file_path, index=True)

# Process each pair at the specified time, ensuring only one trade per day for each pair
def process_pair(config, utc_from, utc_to, global_state):
    logging.info(f"Processing pair: {config['symbol']} for Buy and Sell")

    buyscaler, buymodel, sellscaler, sellmodel = None, None, None, None
    last_trade_day = global_state.get(config['symbol'], {}).get("last_trade_day", None)
    trade_executed_today = global_state.get(config['symbol'], {}).get("trade_executed_today", False)

    # Load models and scalers
    try:
        buyscaler = load(config["buy_scaler_path"])
        buymodel = load(config["buy_model_path"])
    except Exception as e:
        logging.warning(f"Buy model or scaler missing for {config['symbol']}: {e}")
    
    try:
        if "sell_scaler_path" in config and "sell_model_path" in config:
            sellscaler = load(config["sell_scaler_path"])
            sellmodel = load(config["sell_model_path"])
        else:
            logging.warning(f"Sell model or scaler missing for {config['symbol']}")
    except Exception as e:
        logging.warning(f"Error loading sell model for {config['symbol']}: {e}")

    # Load feature lists
    buy_features, sell_features = [], []
    try:
        if "buy_features_path" in config:
            with open(config["buy_features_path"], 'r') as f:
                buy_features = json.load(f)
        if "sell_features_path" in config:
            with open(config["sell_features_path"], 'r') as f:
                sell_features = json.load(f)
    except Exception as e:
        logging.error(f"Error loading features for {config['symbol']}: {e}")

    trade_time = config.get("trade_time", {"hour": 10, "minute": 0})

    try:
        while True:
            now = datetime.now(pytz.utc)
            today = now.date()

            # Reset the flag if it's a new day
            if last_trade_day != today:
                global_state[config['symbol']] = {"trade_executed_today": False, "last_trade_day": today}
                trade_executed_today = False
                last_trade_day = today

            # Only check for trades at the specified time and if no trade was executed today for this pair
            if now.hour == trade_time["hour"] and now.minute == trade_time["minute"] and not trade_executed_today:
                logging.info(f"Checking predictions for {config['symbol']} at {now}")

                # Fetch and process data for Buy and Sell predictions
                df_processed = fetch_and_process_data(config["symbol"], config["timeframe"], ['WILLR_15', 'WILLR_42'], buy_features + sell_features, utc_from, utc_to)
                df_processed = df_processed.shift(periods=1, axis=0).dropna()

                # Make Buy and Sell predictions
                buy_signal = 0
                sell_signal = 0

                if buymodel is not None and buyscaler is not None:
                    scaled_buy_data = buyscaler.transform(df_processed[buy_features])
                    buy_probas = buymodel.predict_proba(scaled_buy_data)[:, 1]
                    buy_signal = apply_custom_threshold(buy_probas, config.get("buy_threshold", 0.5))[-1]  # Last signal

                if sellmodel is not None and sellscaler is not None:
                    scaled_sell_data = sellscaler.transform(df_processed[sell_features])
                    sell_probas = sellmodel.predict_proba(scaled_sell_data)[:, 1]
                    sell_signal = apply_custom_threshold(sell_probas, config.get("sell_threshold", 0.5))[-1]  # Last signal

                # Check for simultaneous Buy and Sell
                if buy_signal == 1 and sell_signal == 1:
                    logging.info(f"Both Buy and Sell signals are 1 for {config['symbol']}. No trade placed.")
                elif buy_signal == 1:
                    logging.info(f"Buy signal generated for {config['symbol']}. Placing Buy trade.")
                    entry_price = mt5.symbol_info_tick(config["symbol"]).ask
                    sl_price, tp_price = calculate_prices(entry_price, config["buy_risk_reward_ratio"], config["mean_candle_size"])
                    place_order(config["symbol"], config["volume"], sl_price, tp_price, "Buy")
                    global_state[config['symbol']]["trade_executed_today"] = True  # Mark that a trade was executed
                elif sell_signal == 1:
                    logging.info(f"Sell signal generated for {config['symbol']}. Placing Sell trade.")
                    entry_price = mt5.symbol_info_tick(config["symbol"]).bid
                    sl_price, tp_price = calculate_prices(entry_price, config["sell_risk_reward_ratio"], config["mean_candle_size"])
                    place_order(config["symbol"], config["volume"], sl_price, tp_price, "Sell")
                    global_state[config['symbol']]["trade_executed_today"] = True  # Mark that a trade was executed

                time.sleep(60)  # Wait for a minute to avoid multiple checks within the same minute
            else:
                time.sleep(30)  # Sleep for 30 seconds before checking again

    except KeyboardInterrupt:
        logging.info(f"Script terminated by user for {config['symbol']}.")
    finally:
        mt5.shutdown()
        logging.info(f"MetaTrader 5 connection closed for {config['symbol']}.")

# Main function
def main():
    config = {'login': 51708234, 'password': '4bM&wuVJcBTnjV', 'server': 'ICMarketsEU-Demo'}
    init_mt5_connection(config['login'], config['password'], config['server'])
    
    gbpusd_config = {
        "symbol": "GBPUSD",
        "timeframe": mt5.TIMEFRAME_D1,
        "volume": 0.1,
        "mean_candle_size": 0.013,
        "buy_risk_reward_ratio": "1:1",
        "sell_risk_reward_ratio": "1:1",
        "buy_scaler_path": 'GBPUSD_D1_3112buy/scaler.joblib',
        "buy_model_path": 'GBPUSD_D1_3112buy/model.joblib',
        "sell_scaler_path": 'GBPUSD_D1_3112sell/scaler.joblib',
        "sell_model_path": 'GBPUSD_D1_3112sell/model.joblib',
        "buy_features_path": 'GBPUSD_D1_3112buy/feature_names.json',
        "sell_features_path": 'GBPUSD_D1_3112sell/feature_names.json',
        "trade_time": {"hour": 10, "minute": 41} 
    }

    usdcad_config = {
        "symbol": "USDCAD",
        "timeframe": mt5.TIMEFRAME_D1,
        "volume": 0.1,
        "mean_candle_size": 0.0088,
        "buy_risk_reward_ratio": "2:3",
        "sell_risk_reward_ratio": "2:3",
        "buy_scaler_path": 'USDCAD_D1_3112_BuyNEW/scaler.joblib',
        "buy_model_path": 'USDCAD_D1_3112_BuyNEW/model.joblib',
        "sell_scaler_path": 'USDCAD_D1_3112_Sell/scaler.joblib',
        "sell_model_path": 'USDCAD_D1_3112_Sell/model.joblib',
        "buy_features_path": 'USDCAD_D1_3112_BuyNEW/feature_names.json',
        "sell_features_path": 'USDCAD_D1_3112_Sell/feature_names.json',
        "trade_time": {"hour": 10, "minute": 41} 
    }

    configs = [gbpusd_config, usdcad_config]

    utc_from = datetime(2023, 5, 1, tzinfo=pytz.utc)
    utc_to = datetime.now(pytz.utc)

    # Global state for trade execution
    global_state = {}

    threads = []
    for config in configs:
        # Initialize state for each symbol
        global_state[config['symbol']] = {"trade_executed_today": False, "last_trade_day": None}

        # Start a new thread for each pair
        thread = threading.Thread(target=process_pair, args=(config, utc_from, utc_to, global_state))
        thread.start()
        threads.append(thread)

    for thread in threads:
        thread.join()

if __name__ == "__main__":
    main()


In [ ]:
import json
import sys
import MetaTrader5 as mt5
import pandas as pd
from datetime import datetime
import pytz
import time
from joblib import load
import logging
from tsfresh import extract_features
from tsfresh.utilities.dataframe_functions import roll_time_series, impute
import talib
import os
import threading

logging.basicConfig(filename='trading_GBPUSD_USDCAD_Buy_Sell_D1.log', level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Initialize connection to MetaTrader 5
def init_mt5_connection(login, password, server):
    if not mt5.initialize(login=login, password=password, server=server):
        logging.error(f"initialize() failed, error code = {mt5.last_error()}")
        sys.exit()
    logging.info("Connected to MetaTrader 5")
    print("Connected to MetaTrader 5")

# Fetch historical data
def fetch_historical_data(symbol, timeframe, start_date, end_date):
    logging.info(f"Fetching historical data for {symbol}")
    print(f"Fetching historical data for {symbol}")
    data = mt5.copy_rates_range(symbol, timeframe, start_date, end_date)
    ohlc_data = pd.DataFrame(data)
    ohlc_data['time'] = pd.to_datetime(ohlc_data['time'], unit='s')
    return ohlc_data[['time', 'open', 'high', 'low', 'close']]

# Add rolling features
def add_rolling_features(df, window):
    logging.info("Adding rolling features")
    df['rolling_mean_open'] = df['open'].rolling(window=window).mean()
    df['rolling_std_open'] = df['open'].rolling(window=window).std()
    df['rolling_mean_close'] = df['close'].rolling(window=window).mean()
    df['rolling_std_close'] = df['close'].rolling(window=window).std()
    df['rolling_mean_high'] = df['high'].rolling(window=window).mean()
    df['rolling_std_high'] = df['high'].rolling(window=window).std()
    df['rolling_mean_low'] = df['low'].rolling(window=window).mean()
    df['rolling_std_low'] = df['low'].rolling(window=window).std()
    return df

# Add lag features
def add_lag_features(df, lags):
    logging.info("Adding lag features")
    for lag in lags:
        df[f'open_lag_{lag}'] = df['open'].shift(lag)
        df[f'close_lag_{lag}'] = df['close'].shift(lag)
        df[f'high_lag_{lag}'] = df['high'].shift(lag)
        df[f'low_lag_{lag}'] = df['low'].shift(lag)
    return df

# Calculate indicators
def calculate_indicators(df):
    logging.info("Calculating indicators")
    for period in [15, 23, 42, 145]:
        df[f'WILLR_{period}'] = talib.WILLR(df['high'], df['low'], df['close'], timeperiod=period)
    return df

# Process data for feature extraction
def process_data_for_features(df, symbol, signals, selected_features):
    logging.info(f"Processing data for feature extraction for {symbol}")
    X_combined = pd.DataFrame(index=df['time'])
    
    for signal in signals:
        df_melted = df[['time', signal]].copy()
        df_melted["Symbols"] = symbol
        df_rolled = roll_time_series(df_melted, column_id="Symbols", column_sort="time",
                                     max_timeshift=20, min_timeshift=5)
        X = extract_features(df_rolled.drop("Symbols", axis=1), column_id="id", column_sort="time", 
                             column_value=signal, impute_function=impute, show_warnings=False)
        X = X.set_index(X.index.map(lambda x: x[1]), drop=True)
        
        # Filter features
        selected_features_X = [feature for feature in selected_features if feature in X.columns]
        X_filtered = X[selected_features_X]
        
        X_combined = X_combined.merge(X_filtered, left_index=True, right_index=True, how='left')
        X_combined = X_combined.dropna()
    return X_combined

# Combine buy/sell data
def fetch_and_process_data(symbol, timeframe, signals, selected_features, start_date, end_date):
    logging.info(f"Fetching and processing data for {symbol}")
    print(f"Fetching and processing data for {symbol}")
    
    df = fetch_historical_data(symbol, timeframe, start_date, end_date)
    df = calculate_indicators(df)
    df = add_rolling_features(df, window=5)
    df = add_lag_features(df, lags=[1, 2, 3, 4, 5])
    
    df = df.dropna().reset_index(drop=True)
    
    features = process_data_for_features(df, symbol, signals, selected_features)
    df = df.set_index('time')
    combined_df = df.merge(features, left_index=True, right_index=True, how='left')
    
    # Handle missing features by filling with zeros
    missing_features = [feat for feat in selected_features if feat not in combined_df.columns]
    for feat in missing_features:
        combined_df[feat] = 0
    
    combined_df = combined_df[selected_features]
    
    combined_df = combined_df.dropna()
    
    return combined_df

# Calculate SL and TP based on entry price and specified percentages
def calculate_prices(entry_price, risk_reward_ratio, mean_candle_size):
    logging.info("Calculating SL and TP prices")
    risk_part, reward_part = map(int, risk_reward_ratio.split(':'))
    risk_amount = mean_candle_size * risk_part
    reward_amount = mean_candle_size * reward_part
    sl_price = entry_price - risk_amount
    tp_price = entry_price + reward_amount
    return sl_price, tp_price

# Place order
def place_order(symbol, volume, sl_price, tp_price, trade_type):
    logging.info(f"Placing {trade_type} order for {symbol}")
    
    # Check if the market is open for the symbol
    symbol_info = mt5.symbol_info(symbol)
    if symbol_info is None:
        logging.error(f"Failed to get symbol info for {symbol}.")
        return
    
    if not symbol_info.visible or symbol_info.trade_mode == mt5.SYMBOL_TRADE_MODE_DISABLED:
        logging.error(f"Market for {symbol} is currently closed or not available for trading.")
        print(f"Market for {symbol} is closed. Cannot place order.")
        return

    # Proceed with order placement
    price = mt5.symbol_info_tick(symbol).ask if trade_type == "Buy" else mt5.symbol_info_tick(symbol).bid
    order_type = mt5.ORDER_TYPE_BUY if trade_type == "Buy" else mt5.ORDER_TYPE_SELL
    
    request = {
        "action": mt5.TRADE_ACTION_DEAL,
        "symbol": symbol,
        "volume": volume,
        "type": order_type,
        "price": price,
        "sl": sl_price,
        "tp": tp_price,
        "deviation": 10,
        "magic": 0,
        "comment": f"Python script {trade_type} order",
        "type_time": mt5.ORDER_TIME_GTC,
        "type_filling": mt5.ORDER_FILLING_IOC,
    }
    
    result = mt5.order_send(request)
    
    if result and result.retcode == mt5.TRADE_RETCODE_DONE:
        logging.info(f"{trade_type} order placed successfully for {symbol}.")
        print(f"{trade_type} order placed successfully for {symbol}.")
    else:
        logging.error(f"Order placement failed for {symbol}: {result.retcode if result else 'No response'} - {mt5.last_error()}")
        print(f"Order placement failed for {symbol}: {result.retcode if result else 'No response'} - {mt5.last_error()}")

# Function to track predictions for both buy and sell to avoid simultaneous trades
def update_trade_decision(symbol, trade_type, pred, trade_decisions):
    if trade_type == "Buy":
        trade_decisions[symbol]['buy'] = pred
    elif trade_type == "Sell":
        trade_decisions[symbol]['sell'] = pred

    # If both buy and sell are 1, no trade should be taken
    if trade_decisions[symbol]['buy'] == 1 and trade_decisions[symbol]['sell'] == 1:
        logging.info(f"Both Buy and Sell predictions are 1 for {symbol}. No trade will be taken.")
        print(f"Both Buy and Sell predictions are 1 for {symbol}. No trade will be taken.")
        return False
    return True

# Predict and trade for Buy or Sell with check for simultaneous signals
def predict_and_trade(scaler, model, symbol, volume, timeframe, risk_reward_ratio, mean_candle_size, start_date, end_date, features, trade_type, trade_decisions, threshold=None):
    logging.info(f"Starting prediction and trading process for {symbol} - {trade_type}")
    
    signals = ['WILLR_15', 'WILLR_42']  # Signals used for both buy and sell

    # Fetch and process data
    df_processed = fetch_and_process_data(symbol, timeframe, signals, features, start_date, end_date)

    # Shift the data
    original_index = df_processed.index
    df_processed = df_processed.shift(periods=1, axis=0).dropna()

    # Align the features with what the scaler expects
    df_processed = df_processed[features]

    # Scale the data
    scaled_data = scaler.transform(df_processed)
    
    # Predict probabilities and apply the provided threshold or default to 0.5
    probas = model.predict_proba(scaled_data)[:, 1]
    custom_threshold = threshold if threshold is not None else 0.5  # Use provided threshold or default to 0.5
    y_pred = (probas >= custom_threshold).astype(int)
    
    # Update trade decisions for the symbol
    pred_decision = 1 if len(y_pred) > 1 and y_pred[-1] == 1 else 0
    should_trade = update_trade_decision(symbol, trade_type, pred_decision, trade_decisions)
    
    if not should_trade:
        logging.info(f"Skipping trade for {symbol} due to both Buy and Sell signals being active.")
        return

    if pred_decision == 1:  # If the prediction is a signal
        entry_price = mt5.symbol_info_tick(symbol).ask if trade_type == "Buy" else mt5.symbol_info_tick(symbol).bid
        sl_price, tp_price = calculate_prices(entry_price, risk_reward_ratio, mean_candle_size)
        place_order(symbol, volume, sl_price, tp_price, trade_type)
    else:
        logging.info(f"No {trade_type} signal generated for {symbol}. Doing nothing.")
        print(f"No {trade_type} signal generated for {symbol}. Doing nothing.")

# Function to process each pair for both buy and sell models without checking new candle updates
def process_pair(config, utc_from, utc_to, trade_decisions):
    logging.info(f"Processing pair: {config['symbol']} for Buy and Sell")

    # Initialize variables for model/scaler availability
    buyscaler, buymodel, sellscaler, sellmodel = None, None, None, None

    # Try to load buy model and scaler
    try:
        buyscaler = load(config["buy_scaler_path"])
        buymodel = load(config["buy_model_path"])
        
        # Load buy features
        with open(config["buy_features_path"], 'r') as f:
            buy_features = json.load(f)
    except Exception as e:
        logging.warning(f"Buy model or scaler missing for {config['symbol']}: {e}")
        print(f"Buy model or scaler missing for {config['symbol']}: {e}")

    # Try to load sell model and scaler
    try:
        if "sell_scaler_path" in config and "sell_model_path" in config:
            sellscaler = load(config["sell_scaler_path"])
            sellmodel = load(config["sell_model_path"])
            
            # Load sell features
            with open(config["sell_features_path"], 'r') as f:
                sell_features = json.load(f)
        else:
            logging.warning(f"Sell model or scaler missing for {config['symbol']}")
            print(f"Sell model or scaler missing for {config['symbol']}")
    except Exception as e:
        logging.warning(f"Error loading sell model for {config['symbol']}: {e}")
        print(f"Error loading sell model for {config['symbol']}: {e}")

    try:
        while True:
            # Get the current UTC time
            current_time = datetime.now(pytz.utc)

            # Check if it's 00:20 UTC
            if current_time.hour == 12 and current_time.minute == 20:
                # Predict and trade for Buy if model is available
                if buyscaler is not None and buymodel is not None:
                    predict_and_trade(
                        buyscaler, buymodel, config["symbol"], config["volume"], config["timeframe"],
                        config["buy_risk_reward_ratio"], config["mean_candle_size"], utc_from, utc_to, buy_features, "Buy",
                        trade_decisions, threshold=config.get("buy_threshold", None)
                    )
                
                # Predict and trade for Sell if model is available
                if sellscaler is not None and sellmodel is not None:
                    predict_and_trade(
                        sellscaler, sellmodel, config["symbol"], config["volume"], config["timeframe"],
                        config["sell_risk_reward_ratio"], config["mean_candle_size"], utc_from, utc_to, sell_features, "Sell",
                        trade_decisions, threshold=config.get("sell_threshold", None)
                    )
                
                logging.info(f"Finished trading for {config['symbol']} at 00:20 UTC")

            # Sleep for 60 seconds and check again
            time.sleep(60)
    except KeyboardInterrupt:
        logging.info(f"Script terminated by user for {config['symbol']}.")
        print(f"Script terminated by user for {config['symbol']}.")
    finally:
        mt5.shutdown()
        logging.info(f"MetaTrader 5 connection closed for {config['symbol']}.")
        print(f"MetaTrader 5 connection closed for {config['symbol']}.")

# Main function
def main():
    # Sample configuration, replace with your actual configuration details
    config = {'login': 51708234, 'password': '4bM&wuVJcBTnjV', 'server': 'ICMarketsEU-Demo'}

    init_mt5_connection(config['login'], config['password'], config['server'])
    
    # Configuration for GBPUSD (Only Buy model provided, no threshold specified)
    gbpusd_config = {
        "symbol": "GBPUSD",
        "timeframe": mt5.TIMEFRAME_D1,
        "volume": 0.1,
        "mean_candle_size": 0.013,
        "buy_risk_reward_ratio": "1:1",
        "sell_risk_reward_ratio": "1:1",
        "buy_scaler_path": 'GBPUSD_D1_3112buy/scaler.joblib',
        "buy_model_path": 'GBPUSD_D1_3112buy/model.joblib',
        "sell_scaler_path": 'GBPUSD_D1_3112sell/scaler.joblib',
        "sell_model_path": 'GBPUSD_D1_3112sell/model.joblib',
        "buy_features_path": 'GBPUSD_D1_3112buy/feature_names.json',
        "sell_features_path": 'GBPUSD_D1_3112sell/feature_names.json'
    }

    # Configuration for USDCAD (Both Buy and Sell models provided, no thresholds specified)
    usdcad_config = {
        "symbol": "USDCAD",
        "timeframe": mt5.TIMEFRAME_D1,
        "volume": 0.1,
        "mean_candle_size": 0.0088,
        "buy_risk_reward_ratio": "2:3",
        "sell_risk_reward_ratio": "2:3",
        "buy_scaler_path": 'USDCAD_D1_3112_BuyNEW/scaler.joblib',
        "buy_model_path": 'USDCAD_D1_3112_BuyNEW/model.joblib',
        "sell_scaler_path": 'USDCAD_D1_3112_Sell/scaler.joblib',
        "sell_model_path": 'USDCAD_D1_3112_Sell/model.joblib',
        "buy_features_path": 'USDCAD_D1_3112_BuyNEW/feature_names.json',
        "sell_features_path": 'USDCAD_D1_3112_Sell/feature_names.json'
    }
    eurusd_config = {
        "symbol": "EURUSD",
        "timeframe": mt5.TIMEFRAME_D1,
        "volume": 0.1,
        "mean_candle_size": 0.0105,
        "buy_risk_reward_ratio": "2:3",
        "buy_scaler_path": 'EURUSD_D1_3112buy/scaler.joblib',
        "buy_model_path": 'EURUSD_D1_3112buy/model.joblib',
        "buy_features_path": 'EURUSD_D1_3112buy/feature_names.json',
    }

    # Load the configurations
    configs = [gbpusd_config, usdcad_config, eurusd_config]

    utc_from = datetime(2023, 5, 1, tzinfo=pytz.utc)
    utc_to = datetime.now(pytz.utc)

    # Create a dictionary to store trade decisions for each symbol
    trade_decisions = {config['symbol']: {'buy': 0, 'sell': 0} for config in configs}

    threads = []
    for config in configs:
        thread = threading.Thread(target=process_pair, args=(config, utc_from, utc_to, trade_decisions))
        thread.start()
        threads.append(thread)

    for thread in threads:
        thread.join()

if __name__ == "__main__":
    main()

In [ ]:
import json
import sys
import MetaTrader5 as mt5
import pandas as pd
from datetime import datetime
import pytz
import time
from joblib import load
import logging
from tsfresh import extract_features
from tsfresh.utilities.dataframe_functions import roll_time_series, impute
import talib
import os
import threading
import subprocess

logging.basicConfig(filename='trading_EURUSD_GBPUSD_USDCAD_Buy_Sell_D1.log', level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Initialize connection to MetaTrader 5
def init_mt5_connection(login, password, server, terminal_path):
    # Start MetaTrader 5 terminal
    subprocess.Popen([terminal_path, '/login', str(login), '/password', password, '/server', server])
    time.sleep(10)  # Give time for terminal to initialize
    if not mt5.initialize(login=login, password=password, server=server):
        logging.error(f"initialize() failed, error code = {mt5.last_error()}")
        sys.exit()
    logging.info("Connected to MetaTrader 5")
    print("Connected to MetaTrader 5")

# Fetch historical data
def fetch_historical_data(symbol, timeframe, start_date, end_date):
    logging.info(f"Fetching historical data for {symbol}")
    print(f"Fetching historical data for {symbol}")
    data = mt5.copy_rates_range(symbol, timeframe, start_date, end_date)
    ohlc_data = pd.DataFrame(data)
    ohlc_data['time'] = pd.to_datetime(ohlc_data['time'], unit='s')
    return ohlc_data[['time', 'open', 'high', 'low', 'close']]

# Add rolling features
def add_rolling_features(df, window):
    #logging.info("Adding rolling features")
    df['rolling_mean_open'] = df['open'].rolling(window=window).mean()
    df['rolling_std_open'] = df['open'].rolling(window=window).std()
    df['rolling_mean_close'] = df['close'].rolling(window=window).mean()
    df['rolling_std_close'] = df['close'].rolling(window=window).std()
    df['rolling_mean_high'] = df['high'].rolling(window=window).mean()
    df['rolling_std_high'] = df['high'].rolling(window=window).std()
    df['rolling_mean_low'] = df['low'].rolling(window=window).mean()
    df['rolling_std_low'] = df['low'].rolling(window=window).std()
    return df

# Add lag features
def add_lag_features(df, lags):
    #logging.info("Adding lag features")
    for lag in lags:
        df[f'open_lag_{lag}'] = df['open'].shift(lag)
        df[f'close_lag_{lag}'] = df['close'].shift(lag)
        df[f'high_lag_{lag}'] = df['high'].shift(lag)
        df[f'low_lag_{lag}'] = df['low'].shift(lag)
    return df

# Calculate indicators
def calculate_indicators(df):
    #logging.info("Calculating indicators")
    for period in [15, 23, 42, 145]:
        df[f'WILLR_{period}'] = talib.WILLR(df['high'], df['low'], df['close'], timeperiod=period)
    return df

# Process data for feature extraction
def process_data_for_features(df, symbol, signals, selected_features):
    #logging.info(f"Processing data for feature extraction for {symbol}")
    X_combined = pd.DataFrame(index=df['time'])
    
    for signal in signals:
        df_melted = df[['time', signal]].copy()
        df_melted["Symbols"] = symbol
        df_rolled = roll_time_series(df_melted, column_id="Symbols", column_sort="time",
                                     max_timeshift=20, min_timeshift=5)
        X = extract_features(df_rolled.drop("Symbols", axis=1), column_id="id", column_sort="time", 
                             column_value=signal, impute_function=impute, show_warnings=False)
        X = X.set_index(X.index.map(lambda x: x[1]), drop=True)
        
        # Filter features
        selected_features_X = [feature for feature in selected_features if feature in X.columns]
        X_filtered = X[selected_features_X]
        
        X_combined = X_combined.merge(X_filtered, left_index=True, right_index=True, how='left')
        X_combined = X_combined.dropna()
    return X_combined

# Combine buy/sell data
def fetch_and_process_data(symbol, timeframe, signals, selected_features, start_date, end_date):
    logging.info(f"Fetching and processing data for {symbol}")
    print(f"Fetching and processing data for {symbol}")
    
    df = fetch_historical_data(symbol, timeframe, start_date, end_date)
    df = calculate_indicators(df)
    df = add_rolling_features(df, window=5)
    df = add_lag_features(df, lags=[1, 2, 3, 4, 5])
    
    df = df.dropna().reset_index(drop=True)
    
    features = process_data_for_features(df, symbol, signals, selected_features)
    df = df.set_index('time')
    combined_df = df.merge(features, left_index=True, right_index=True, how='left')
    
    # Handle missing features by filling with zeros
    missing_features = [feat for feat in selected_features if feat not in combined_df.columns]
    for feat in missing_features:
        combined_df[feat] = 0
    
    combined_df = combined_df[selected_features]
    
    combined_df = combined_df.dropna()
    
    return combined_df

# Calculate SL and TP based on entry price and specified percentages
def calculate_prices(entry_price, risk_reward_ratio, mean_candle_size):
    #logging.info("Calculating SL and TP prices")
    risk_part, reward_part = map(int, risk_reward_ratio.split(':'))
    risk_amount = mean_candle_size * risk_part
    reward_amount = mean_candle_size * reward_part
    sl_price = entry_price - risk_amount
    tp_price = entry_price + reward_amount
    return sl_price, tp_price

# Place order
def place_order(symbol, volume, sl_price, tp_price, trade_type):
    logging.info(f"Placing {trade_type} order for {symbol}")
    
    # Check if the market is open for the symbol
    symbol_info = mt5.symbol_info(symbol)
    if symbol_info is None:
        logging.error(f"Failed to get symbol info for {symbol}.")
        return
    
    if not symbol_info.visible or symbol_info.trade_mode == mt5.SYMBOL_TRADE_MODE_DISABLED:
        logging.error(f"Market for {symbol} is currently closed or not available for trading.")
        print(f"Market for {symbol} is closed. Cannot place order.")
        return

    # Proceed with order placement
    price = mt5.symbol_info_tick(symbol).ask if trade_type == "Buy" else mt5.symbol_info_tick(symbol).bid
    order_type = mt5.ORDER_TYPE_BUY if trade_type == "Buy" else mt5.ORDER_TYPE_SELL
    
    request = {
        "action": mt5.TRADE_ACTION_DEAL,
        "symbol": symbol,
        "volume": volume,
        "type": order_type,
        "price": price,
        "sl": sl_price,
        "tp": tp_price,
        "deviation": 10,
        "magic": 0,
        "comment": f"Python script {trade_type} order",
        "type_time": mt5.ORDER_TIME_GTC,
        "type_filling": mt5.ORDER_FILLING_IOC,
    }
    
    result = mt5.order_send(request)
    
    if result and result.retcode == mt5.TRADE_RETCODE_DONE:
        logging.info(f"{trade_type} order placed successfully for {symbol}.")
        print(f"{trade_type} order placed successfully for {symbol}.")
    else:
        logging.error(f"Order placement failed for {symbol}: {result.retcode if result else 'No response'} - {mt5.last_error()}")
        print(f"Order placement failed for {symbol}: {result.retcode if result else 'No response'} - {mt5.last_error()}")

# Function to track predictions for both buy and sell to avoid simultaneous trades
def update_trade_decision(symbol, trade_type, pred, trade_decisions):
    if trade_type == "Buy":
        trade_decisions[symbol]['buy'] = pred
    elif trade_type == "Sell":
        trade_decisions[symbol]['sell'] = pred

    # If both buy and sell are 1, no trade should be taken
    if trade_decisions[symbol]['buy'] == 1 and trade_decisions[symbol]['sell'] == 1:
        logging.info(f"Both Buy and Sell predictions are 1 for {symbol}. No trade will be taken.")
        print(f"Both Buy and Sell predictions are 1 for {symbol}. No trade will be taken.")
        return False
    return True

# Predict and trade for Buy or Sell with check for simultaneous signals
def predict_and_trade(scaler, model, symbol, volume, timeframe, risk_reward_ratio, mean_candle_size, start_date, end_date, features, trade_type, trade_decisions, threshold=None):
    #logging.info(f"Starting prediction and trading process for {symbol} - {trade_type}")
    
    signals = ['WILLR_15', 'WILLR_42']  # Signals used for both buy and sell

    # Fetch and process data
    df_processed = fetch_and_process_data(symbol, timeframe, signals, features, start_date, end_date)

    # Shift the data
    original_index = df_processed.index
    df_processed = df_processed.shift(periods=1, axis=0).dropna()

    # Align the features with what the scaler expects
    df_processed = df_processed[features]

    # Scale the data
    scaled_data = scaler.transform(df_processed)
    
    # Predict probabilities and apply the provided threshold or default to 0.5
    probas = model.predict_proba(scaled_data)[:, 1]
    custom_threshold = threshold if threshold is not None else 0.5  # Use provided threshold or default to 0.5
    y_pred = (probas >= custom_threshold).astype(int)
    
    # Update trade decisions for the symbol
    pred_decision = 1 if len(y_pred) > 1 and y_pred[-1] == 1 else 0
    should_trade = update_trade_decision(symbol, trade_type, pred_decision, trade_decisions)
    
    if not should_trade:
        logging.info(f"Skipping trade for {symbol} due to both Buy and Sell signals being active.")
        return

    if pred_decision == 1:  # If the prediction is a signal
        entry_price = mt5.symbol_info_tick(symbol).ask if trade_type == "Buy" else mt5.symbol_info_tick(symbol).bid
        sl_price, tp_price = calculate_prices(entry_price, risk_reward_ratio, mean_candle_size)
        place_order(symbol, volume, sl_price, tp_price, trade_type)
    else:
        logging.info(f"No {trade_type} signal generated for {symbol}. Doing nothing.")
        print(f"No {trade_type} signal generated for {symbol}. Doing nothing.")

# Function to process each pair for both buy and sell models without checking new candle updates
def process_pair(config, utc_from, utc_to, trade_decisions):
    logging.info(f"Processing pair: {config['symbol']} for Buy and Sell")

    # Initialize variables for model/scaler availability
    buyscaler, buymodel, sellscaler, sellmodel = None, None, None, None

    # Try to load buy model and scaler
    try:
        buyscaler = load(config["buy_scaler_path"])
        buymodel = load(config["buy_model_path"])
        
        # Load buy features
        with open(config["buy_features_path"], 'r') as f:
            buy_features = json.load(f)
    except Exception as e:
        logging.warning(f"Buy model or scaler missing for {config['symbol']}: {e}")
        print(f"Buy model or scaler missing for {config['symbol']}: {e}")

    # Try to load sell model and scaler
    try:
        if "sell_scaler_path" in config and "sell_model_path" in config:
            sellscaler = load(config["sell_scaler_path"])
            sellmodel = load(config["sell_model_path"])
            
            # Load sell features
            with open(config["sell_features_path"], 'r') as f:
                sell_features = json.load(f)
        else:
            logging.warning(f"Sell model or scaler missing for {config['symbol']}")
            print(f"Sell model or scaler missing for {config['symbol']}")
    except Exception as e:
        logging.warning(f"Error loading sell model for {config['symbol']}: {e}")
        print(f"Error loading sell model for {config['symbol']}: {e}")

    try:
        init_mt5_connection(config['login'], config['password'], config['server'], config['terminal_path'])
        while True:
            # Get the current UTC time
            current_time = datetime.now(pytz.utc)

            # Check if it's 00:20 UTC
            if current_time.hour == 14 and current_time.minute == 8:
                # Predict and trade for Buy if model is available
                if buyscaler is not None and buymodel is not None:
                    predict_and_trade(
                        buyscaler, buymodel, config["symbol"], config["volume"], config["timeframe"],
                        config["buy_risk_reward_ratio"], config["mean_candle_size"], utc_from, utc_to, buy_features, "Buy",
                        trade_decisions, threshold=config.get("buy_threshold", None)
                    )
                
                # Predict and trade for Sell if model is available
                if sellscaler is not None and sellmodel is not None:
                    predict_and_trade(
                        sellscaler, sellmodel, config["symbol"], config["volume"], config["timeframe"],
                        config["sell_risk_reward_ratio"], config["mean_candle_size"], utc_from, utc_to, sell_features, "Sell",
                        trade_decisions, threshold=config.get("sell_threshold", None)
                    )
                
                logging.info(f"Finished trading for {config['symbol']} at 00:20 UTC")

            # Sleep for 60 seconds and check again
            time.sleep(60)
    except KeyboardInterrupt:
        logging.info(f"Script terminated by user for {config['symbol']}.")
        print(f"Script terminated by user for {config['symbol']}.")
    finally:
        mt5.shutdown()
        logging.info(f"MetaTrader 5 connection closed for {config['symbol']}.")
        print(f"MetaTrader 5 connection closed for {config['symbol']}.")

# Main function
def main():
    # Sample configuration, replace with your actual configuration details
    eurusd_config = {
        "symbol": "EURUSD",
        "login": 51988090,
        "password": '1fMdV52$74EOcw',
        "server": 'ICMarketsEU-Demo',
        "terminal_path": r"C:\Program Files\MetaTrader5ICMarketsEU2\terminal64.exe",
        "timeframe": mt5.TIMEFRAME_D1,
        "volume": 0.1,
        "mean_candle_size": 0.0105,
        "buy_risk_reward_ratio": "2:3",
        "buy_scaler_path": 'EURUSD_D1_3112buy/scaler.joblib',
        "buy_model_path": 'EURUSD_D1_3112buy/model.joblib',
        "buy_features_path": 'EURUSD_D1_3112buy/feature_names.json',
    }

    gbpusd_config = {
        "symbol": "GBPUSD",
        "login": 51988092,
        "password": 'ty!H!03FNx!kEh',
        "server": 'ICMarketsEU-Demo',
        "terminal_path": r"C:\Program Files\MetaTrader5ICMarketsEU3\terminal64.exe",
        "timeframe": mt5.TIMEFRAME_D1,
        "volume": 0.1,
        "mean_candle_size": 0.013,
        "buy_risk_reward_ratio": "1:1",
        "sell_risk_reward_ratio": "1:1",
        "buy_scaler_path": 'GBPUSD_D1_3112buy/scaler.joblib',
        "buy_model_path": 'GBPUSD_D1_3112buy/model.joblib',
        "sell_scaler_path": 'GBPUSD_D1_3112sell/scaler.joblib',
        "sell_model_path": 'GBPUSD_D1_3112sell/model.joblib',
        "buy_features_path": 'GBPUSD_D1_3112buy/feature_names.json',
        "sell_features_path": 'GBPUSD_D1_3112sell/feature_names.json'
    }

    usdcad_config = {
        "symbol": "USDCAD",
        "login": 51988094,
        "password": '!rwj8zZ5iA$Sz5',
        "server": 'ICMarketsEU-Demo',
        "terminal_path": r"C:\Program Files\MetaTrader5ICMarketsEU4\terminal64.exe",
        "timeframe": mt5.TIMEFRAME_D1,
        "volume": 0.1,
        "mean_candle_size": 0.0088,
        "buy_risk_reward_ratio": "2:3",
        "sell_risk_reward_ratio": "2:3",
        "buy_scaler_path": 'USDCAD_D1_3112_BuyNEW/scaler.joblib',
        "buy_model_path": 'USDCAD_D1_3112_BuyNEW/model.joblib',
        "sell_scaler_path": 'USDCAD_D1_3112_Sell/scaler.joblib',
        "sell_model_path": 'USDCAD_D1_3112_Sell/model.joblib',
        "buy_features_path": 'USDCAD_D1_3112_BuyNEW/feature_names.json',
        "sell_features_path": 'USDCAD_D1_3112_Sell/feature_names.json'
    }

    # Load the configurations
    configs = [gbpusd_config, usdcad_config, eurusd_config]

    utc_from = datetime(2023, 5, 1, tzinfo=pytz.utc)
    utc_to = datetime.now(pytz.utc)

    # Create a dictionary to store trade decisions for each symbol
    trade_decisions = {config['symbol']: {'buy': 0, 'sell': 0} for config in configs}

    threads = []
    for config in configs:
        thread = threading.Thread(target=process_pair, args=(config, utc_from, utc_to, trade_decisions))
        thread.start()
        threads.append(thread)

    for thread in threads:
        thread.join()

if __name__ == "__main__":
    main()

In [ ]:
import json
import sys
import MetaTrader5 as mt5
import pandas as pd
from datetime import datetime
import pytz
import time
from joblib import load
import logging
from tsfresh import extract_features
from tsfresh.utilities.dataframe_functions import roll_time_series, impute
import talib
import os
import threading
import subprocess

logging.basicConfig(filename='trading_EURUSD_GBPUSD_USDCAD_Buy_Sell_D1.log', level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Initialize connection to MetaTrader 5
def init_mt5_connection(login, password, server, terminal_path):
    # Start MetaTrader 5 terminal
    subprocess.Popen([terminal_path, '/login', str(login), '/password', password, '/server', server])
    time.sleep(10)  # Give time for terminal to initialize
    if not mt5.initialize(login=login, password=password, server=server):
        logging.error(f"initialize() failed, error code = {mt5.last_error()}")
        sys.exit()
    logging.info("Connected to MetaTrader 5")
    print("Connected to MetaTrader 5")

# Fetch historical data
def fetch_historical_data(symbol, timeframe, start_date, end_date):
    logging.info(f"Fetching historical data for {symbol}")
    print(f"Fetching historical data for {symbol}")
    data = mt5.copy_rates_range(symbol, timeframe, start_date, end_date)
    ohlc_data = pd.DataFrame(data)
    ohlc_data['time'] = pd.to_datetime(ohlc_data['time'], unit='s')
    return ohlc_data[['time', 'open', 'high', 'low', 'close']]

# Calculate mean candle size dynamically
def calculate_mean_candle_size(df):
    df['candle_size'] = df['high'] - df['low']
    mean_candle_size = df['candle_size'].mean()
    return mean_candle_size

# Add rolling features
def add_rolling_features(df, window):
    df['rolling_mean_open'] = df['open'].rolling(window=window).mean()
    df['rolling_std_open'] = df['open'].rolling(window=window).std()
    df['rolling_mean_close'] = df['close'].rolling(window=window).mean()
    df['rolling_std_close'] = df['close'].rolling(window=window).std()
    df['rolling_mean_high'] = df['high'].rolling(window=window).mean()
    df['rolling_std_high'] = df['high'].rolling(window=window).std()
    df['rolling_mean_low'] = df['low'].rolling(window=window).mean()
    df['rolling_std_low'] = df['low'].rolling(window=window).std()
    return df

# Add lag features
def add_lag_features(df, lags):
    for lag in lags:
        df[f'open_lag_{lag}'] = df['open'].shift(lag)
        df[f'close_lag_{lag}'] = df['close'].shift(lag)
        df[f'high_lag_{lag}'] = df['high'].shift(lag)
        df[f'low_lag_{lag}'] = df['low'].shift(lag)
    return df

# Calculate indicators
def calculate_indicators(df):
    for period in [15, 23, 42, 145]:
        df[f'WILLR_{period}'] = talib.WILLR(df['high'], df['low'], df['close'], timeperiod=period)
    return df

# Process data for feature extraction
def process_data_for_features(df, symbol, signals, selected_features):
    X_combined = pd.DataFrame(index=df['time'])
    
    for signal in signals:
        df_melted = df[['time', signal]].copy()
        df_melted["Symbols"] = symbol
        df_rolled = roll_time_series(df_melted, column_id="Symbols", column_sort="time",
                                     max_timeshift=20, min_timeshift=5)
        X = extract_features(df_rolled.drop("Symbols", axis=1), column_id="id", column_sort="time", 
                             column_value=signal, impute_function=impute, show_warnings=False)
        X = X.set_index(X.index.map(lambda x: x[1]), drop=True)
        
        selected_features_X = [feature for feature in selected_features if feature in X.columns]
        X_filtered = X[selected_features_X]
        
        X_combined = X_combined.merge(X_filtered, left_index=True, right_index=True, how='left')
        X_combined = X_combined.dropna()
    return X_combined

# Combine buy/sell data
def fetch_and_process_data(symbol, timeframe, signals, selected_features, start_date, end_date):
    logging.info(f"Fetching and processing data for {symbol}")
    print(f"Fetching and processing data for {symbol}")
    
    df = fetch_historical_data(symbol, timeframe, start_date, end_date)
    
    # Calculate mean candle size before modifying the DataFrame
    mean_candle_size = calculate_mean_candle_size(df)
    
    df = calculate_indicators(df)
    df = add_rolling_features(df, window=5)
    df = add_lag_features(df, lags=[1, 2, 3, 4, 5])
    
    df = df.dropna().reset_index(drop=True)
    
    features = process_data_for_features(df, symbol, signals, selected_features)
    df = df.set_index('time')
    combined_df = df.merge(features, left_index=True, right_index=True, how='left')
    
    missing_features = [feat for feat in selected_features if feat not in combined_df.columns]
    for feat in missing_features:
        combined_df[feat] = 0
    
    combined_df = combined_df[selected_features]
    
    combined_df = combined_df.dropna()
    
    return combined_df, mean_candle_size

# Calculate SL and TP based on entry price and specified percentages
def calculate_prices(entry_price, risk_reward_ratio, mean_candle_size):
    risk_part, reward_part = map(int, risk_reward_ratio.split(':'))
    risk_amount = mean_candle_size * risk_part
    reward_amount = mean_candle_size * reward_part
    sl_price = entry_price - risk_amount
    tp_price = entry_price + reward_amount
    return sl_price, tp_price

# Place order
def place_order(symbol, volume, sl_price, tp_price, trade_type):
    logging.info(f"Placing {trade_type} order for {symbol}")
    
    # Check if the market is open for the symbol
    symbol_info = mt5.symbol_info(symbol)
    if symbol_info is None:
        logging.error(f"Failed to get symbol info for {symbol}.")
        return
    
    if not symbol_info.visible or symbol_info.trade_mode == mt5.SYMBOL_TRADE_MODE_DISABLED:
        logging.error(f"Market for {symbol} is currently closed or not available for trading.")
        print(f"Market for {symbol} is closed. Cannot place order.")
        return

    # Proceed with order placement
    price = mt5.symbol_info_tick(symbol).ask if trade_type == "Buy" else mt5.symbol_info_tick(symbol).bid
    order_type = mt5.ORDER_TYPE_BUY if trade_type == "Buy" else mt5.ORDER_TYPE_SELL
    
    request = {
        "action": mt5.TRADE_ACTION_DEAL,
        "symbol": symbol,
        "volume": volume,
        "type": order_type,
        "price": price,
        "sl": sl_price,
        "tp": tp_price,
        "deviation": 10,
        "magic": 0,
        "comment": f"Python script {trade_type} order",
        "type_time": mt5.ORDER_TIME_GTC,
        "type_filling": mt5.ORDER_FILLING_IOC,
    }
    
    result = mt5.order_send(request)
    
    if result and result.retcode == mt5.TRADE_RETCODE_DONE:
        logging.info(f"{trade_type} order placed successfully for {symbol}.")
        print(f"{trade_type} order placed successfully for {symbol}.")
    else:
        logging.error(f"Order placement failed for {symbol}: {result.retcode if result else 'No response'} - {mt5.last_error()}")
        print(f"Order placement failed for {symbol}: {result.retcode if result else 'No response'} - {mt5.last_error()}")

# Function to track predictions for both buy and sell to avoid simultaneous trades
def update_trade_decision(symbol, trade_type, pred, trade_decisions):
    if trade_type == "Buy":
        trade_decisions[symbol]['buy'] = pred
    elif trade_type == "Sell":
        trade_decisions[symbol]['sell'] = pred

    # If both buy and sell are 1, no trade should be taken
    if trade_decisions[symbol]['buy'] == 1 and trade_decisions[symbol]['sell'] == 1:
        logging.info(f"Both Buy and Sell predictions are 1 for {symbol}. No trade will be taken.")
        print(f"Both Buy and Sell predictions are 1 for {symbol}. No trade will be taken.")
        return False
    return True

# Predict and trade for Buy or Sell with check for simultaneous signals
def predict_and_trade(scaler, model, symbol, volume, timeframe, risk_reward_ratio, start_date, end_date, features, trade_type, trade_decisions, threshold=None):
    signals = ['WILLR_15', 'WILLR_42']  # Signals used for both buy and sell

    # Fetch and process data
    df_processed, mean_candle_size = fetch_and_process_data(symbol, timeframe, signals, features, start_date, end_date)

    # Check if mean_candle_size is None or NaN
    if pd.isna(mean_candle_size) or mean_candle_size == 0:
        logging.error(f"Mean candle size for {symbol} is not valid.")
        print(f"Mean candle size for {symbol} is not valid. Skipping trading.")
        return

    # Shift the data
    original_index = df_processed.index
    df_processed = df_processed.shift(periods=1, axis=0).dropna()

    # Align the features with what the scaler expects
    df_processed = df_processed[features]

    # Scale the data
    scaled_data = scaler.transform(df_processed)
    
    # Predict probabilities and apply the provided threshold or default to 0.5
    probas = model.predict_proba(scaled_data)[:, 1]
    custom_threshold = threshold if threshold is not None else 0.5
    y_pred = (probas >= custom_threshold).astype(int)
    
    # Update trade decisions for the symbol
    pred_decision = 1 if len(y_pred) > 1 and y_pred[-1] == 1 else 0
    should_trade = update_trade_decision(symbol, trade_type, pred_decision, trade_decisions)
    
    if not should_trade:
        logging.info(f"Skipping trade for {symbol} due to both Buy and Sell signals being active.")
        return

    if pred_decision == 1:  # If the prediction is a signal
        entry_price = mt5.symbol_info_tick(symbol).ask if trade_type == "Buy" else mt5.symbol_info_tick(symbol).bid
        sl_price, tp_price = calculate_prices(entry_price, risk_reward_ratio, mean_candle_size)
        place_order(symbol, volume, sl_price, tp_price, trade_type)
    else:
        logging.info(f"No {trade_type} signal generated for {symbol}. Doing nothing.")
        print(f"No {trade_type} signal generated for {symbol}. Doing nothing.")

# Function to process each pair for both buy and sell models without checking new candle updates
def process_pair(config, utc_from, utc_to, trade_decisions):
    logging.info(f"Processing pair: {config['symbol']} for Buy and Sell")

    # Initialize variables for model/scaler availability
    buyscaler, buymodel, sellscaler, sellmodel = None, None, None, None

    # Try to load buy model and scaler
    try:
        buyscaler = load(config["buy_scaler_path"])
        buymodel = load(config["buy_model_path"])
        
        # Load buy features
        with open(config["buy_features_path"], 'r') as f:
            buy_features = json.load(f)
    except Exception as e:
        logging.warning(f"Buy model or scaler missing for {config['symbol']}: {e}")
        print(f"Buy model or scaler missing for {config['symbol']}: {e}")

    # Try to load sell model and scaler
    try:
        if "sell_scaler_path" in config and "sell_model_path" in config:
            sellscaler = load(config["sell_scaler_path"])
            sellmodel = load(config["sell_model_path"])
            
            # Load sell features
            with open(config["sell_features_path"], 'r') as f:
                sell_features = json.load(f)
        else:
            logging.warning(f"Sell model or scaler missing for {config['symbol']}")
            print(f"Sell model or scaler missing for {config['symbol']}")
    except Exception as e:
        logging.warning(f"Error loading sell model for {config['symbol']}: {e}")
        print(f"Error loading sell model for {config['symbol']}: {e}")

    try:
        init_mt5_connection(config['login'], config['password'], config['server'], config['terminal_path'])
        while True:
            # Get the current UTC time
            current_time = datetime.now(pytz.utc)

            # Check if it's 00:20 UTC
            if current_time.hour == 22 and current_time.minute == 35:
                # Predict and trade for Buy if model is available
                if buyscaler is not None and buymodel is not None:
                    predict_and_trade(
                        buyscaler, buymodel, config["symbol"], config["volume"], config["timeframe"],
                        config["buy_risk_reward_ratio"], utc_from, utc_to, buy_features, "Buy",
                        trade_decisions, threshold=config.get("buy_threshold", None)
                    )
                
                # Predict and trade for Sell if model is available
                if sellscaler is not None and sellmodel is not None:
                    predict_and_trade(
                        sellscaler, sellmodel, config["symbol"], config["volume"], config["timeframe"],
                        config["sell_risk_reward_ratio"], utc_from, utc_to, sell_features, "Sell",
                        trade_decisions, threshold=config.get("sell_threshold", None)
                    )
                
                logging.info(f"Finished trading for {config['symbol']} at 00:20 UTC")

            # Sleep for 60 seconds and check again
            time.sleep(60)
    except KeyboardInterrupt:
        logging.info(f"Script terminated by user for {config['symbol']}.")
        print(f"Script terminated by user for {config['symbol']}.")
    finally:
        mt5.shutdown()
        logging.info(f"MetaTrader 5 connection closed for {config['symbol']}.")
        print(f"MetaTrader 5 connection closed for {config['symbol']}.")

# Main function
def main():
    # Sample configuration, replace with your actual configuration details
    eurusd_config = {
        "symbol": "EURUSD",
        "login": 51988090,
        "password": '1fMdV52$74EOcw',
        "server": 'ICMarketsEU-Demo',
        "terminal_path": r"C:\Program Files\MetaTrader5ICMarketsEU2\terminal64.exe",
        "timeframe": mt5.TIMEFRAME_D1,
        "volume": 0.1,
        "buy_risk_reward_ratio": "2:3",
        "buy_scaler_path": 'EURUSD_D1_3112buy/scaler.joblib',
        "buy_model_path": 'EURUSD_D1_3112buy/model.joblib',
        "buy_features_path": 'EURUSD_D1_3112buy/feature_names.json',
    }

    gbpusd_config = {
        "symbol": "GBPUSD",
        "login": 51988092,
        "password": 'ty!H!03FNx!kEh',
        "server": 'ICMarketsEU-Demo',
        "terminal_path": r"C:\Program Files\MetaTrader5ICMarketsEU3\terminal64.exe",
        "timeframe": mt5.TIMEFRAME_D1,
        "volume": 0.1,
        "buy_risk_reward_ratio": "1:1",
        "sell_risk_reward_ratio": "1:1",
        "buy_scaler_path": 'GBPUSD_D1_3112buy/scaler.joblib',
        "buy_model_path": 'GBPUSD_D1_3112buy/model.joblib',
        "sell_scaler_path": 'GBPUSD_D1_3112sell/scaler.joblib',
        "sell_model_path": 'GBPUSD_D1_3112sell/model.joblib',
        "buy_features_path": 'GBPUSD_D1_3112buy/feature_names.json',
        "sell_features_path": 'GBPUSD_D1_3112sell/feature_names.json'
    }

    usdcad_config = {
        "symbol": "USDCAD",
        "login": 51988094,
        "password": '!rwj8zZ5iA$Sz5',
        "server": 'ICMarketsEU-Demo',
        "terminal_path": r"C:\Program Files\MetaTrader5ICMarketsEU4\terminal64.exe",
        "timeframe": mt5.TIMEFRAME_D1,
        "volume": 0.1,
        "buy_risk_reward_ratio": "2:3",
        "sell_risk_reward_ratio": "2:3",
        "buy_scaler_path": 'USDCAD_D1_3112_BuyNEW/scaler.joblib',
        "buy_model_path": 'USDCAD_D1_3112_BuyNEW/model.joblib',
        "sell_scaler_path": 'USDCAD_D1_3112_Sell/scaler.joblib',
        "sell_model_path": 'USDCAD_D1_3112_Sell/model.joblib',
        "buy_features_path": 'USDCAD_D1_3112_BuyNEW/feature_names.json',
        "sell_features_path": 'USDCAD_D1_3112_Sell/feature_names.json'
    }

    # Load the configurations
    configs = [gbpusd_config, usdcad_config, eurusd_config]

    utc_from = datetime(2023, 5, 1, tzinfo=pytz.utc)
    utc_to = datetime.now(pytz.utc)

    # Create a dictionary to store trade decisions for each symbol
    trade_decisions = {config['symbol']: {'buy': 0, 'sell': 0} for config in configs}

    threads = []
    for config in configs:
        thread = threading.Thread(target=process_pair, args=(config, utc_from, utc_to, trade_decisions))
        thread.start()
        threads.append(thread)

    for thread in threads:
        thread.join()

if __name__ == "__main__":
    main()

In [1]:
import json
import sys
import MetaTrader5 as mt5
import pandas as pd
from datetime import datetime
import pytz
import time
from joblib import load
import logging
from tsfresh import extract_features
from tsfresh.utilities.dataframe_functions import roll_time_series, impute
import talib
import os
import threading
import subprocess
import psutil

# Initialize the lock globally
mt5_lock = threading.Lock()

logging.basicConfig(filename='trading_EURUSD_GBPUSD_USDCAD_Buy_Sell_D1.log', level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Kill any existing MetaTrader 5 instances
def kill_mt5_instances():
    for process in psutil.process_iter():
        try:
            if "terminal64.exe" in process.name().lower():
                process.kill()
        except (psutil.NoSuchProcess, psutil.AccessDenied, psutil.ZombieProcess):
            pass  # Ignore these exceptions

# Initialize connection to MetaTrader 5 with proper locking and multiple instances handling
def init_mt5_connection(login, password, server, terminal_path):
    mt5.shutdown()
    time.sleep(5)

    # Start MetaTrader 5 terminal for the specific account
    subprocess.Popen([terminal_path, '/portable', '/login', str(login), '/password', password, '/server', server])
    time.sleep(10)

    for _ in range(30):  # Poll for 30 seconds, check if MT5 is running
        if mt5.initialize(login=login, password=password, server=server):
            logging.info(f"Connected to MetaTrader 5 with login: {login}")
            print(f"Connected to MetaTrader 5 with login: {login}")
            return True
        time.sleep(1)

    logging.error(f"initialize() failed, error code = {mt5.last_error()}")
    sys.exit()

# Fetch historical data
def fetch_historical_data(symbol, timeframe, start_date, end_date):
    logging.info(f"Fetching historical data for {symbol}")
    print(f"Fetching historical data for {symbol}")
    data = mt5.copy_rates_range(symbol, timeframe, start_date, end_date)
    
    # Check if data is empty or None
    if data is None or len(data) == 0:
        logging.error(f"No historical data found for {symbol} in the given date range.")
        print(f"No historical data found for {symbol}.")
        return None
    
    ohlc_data = pd.DataFrame(data)
    if 'time' not in ohlc_data.columns:
        logging.error(f"'time' column missing in the fetched data for {symbol}.")
        print(f"Error: 'time' column missing in the fetched data for {symbol}.")
        return None

    ohlc_data['time'] = pd.to_datetime(ohlc_data['time'], unit='s')
    return ohlc_data[['time', 'open', 'high', 'low', 'close']]

# Calculate mean candle size dynamically
def calculate_mean_candle_size(df):
    df['candle_size'] = df['high'] - df['low']
    mean_candle_size = df['candle_size'].mean()
    return mean_candle_size

# Add rolling features
def add_rolling_features(df, window):
    df['rolling_mean_open'] = df['open'].rolling(window=window).mean()
    df['rolling_std_open'] = df['open'].rolling(window=window).std()
    df['rolling_mean_close'] = df['close'].rolling(window=window).mean()
    df['rolling_std_close'] = df['close'].rolling(window=window).std()
    df['rolling_mean_high'] = df['high'].rolling(window=window).mean()
    df['rolling_std_high'] = df['high'].rolling(window=window).std()
    df['rolling_mean_low'] = df['low'].rolling(window=window).mean()
    df['rolling_std_low'] = df['low'].rolling(window=window).std()
    return df

# Add lag features
def add_lag_features(df, lags):
    for lag in lags:
        df[f'open_lag_{lag}'] = df['open'].shift(lag)
        df[f'close_lag_{lag}'] = df['close'].shift(lag)
        df[f'high_lag_{lag}'] = df['high'].shift(lag)
        df[f'low_lag_{lag}'] = df['low'].shift(lag)
    return df

# Calculate indicators
def calculate_indicators(df):
    for period in [15, 23, 42, 145]:
        df[f'WILLR_{period}'] = talib.WILLR(df['high'], df['low'], df['close'], timeperiod=period)
    return df

# Process data for feature extraction
def process_data_for_features(df, symbol, signals, selected_features):
    X_combined = pd.DataFrame(index=df['time'])

    for signal in signals:
        df_melted = df[['time', signal]].copy()
        df_melted["Symbols"] = symbol
        df_rolled = roll_time_series(df_melted, column_id="Symbols", column_sort="time", max_timeshift=20, min_timeshift=5)
        X = extract_features(df_rolled.drop("Symbols", axis=1), column_id="id", column_sort="time", column_value=signal, impute_function=impute, show_warnings=False)
        X = X.set_index(X.index.map(lambda x: x[1]), drop=True)

        selected_features_X = [feature for feature in selected_features if feature in X.columns]
        X_filtered = X[selected_features_X]

        X_combined = X_combined.merge(X_filtered, left_index=True, right_index=True, how='left')
        X_combined = X_combined.dropna()
    return X_combined

# Combine buy/sell data
def fetch_and_process_data(symbol, timeframe, signals, selected_features, start_date, end_date):
    logging.info(f"Fetching and processing data for {symbol}")
    print(f"Fetching and processing data for {symbol}")

    df = fetch_historical_data(symbol, timeframe, start_date, end_date)
    
    if df is None:  # If no data is fetched, exit
        return None, None

    # Calculate mean candle size before modifying the DataFrame
    mean_candle_size = calculate_mean_candle_size(df)

    df = calculate_indicators(df)
    df = add_rolling_features(df, window=5)
    df = add_lag_features(df, lags=[1, 2, 3, 4, 5])

    df = df.dropna().reset_index(drop=True)

    features = process_data_for_features(df, symbol, signals, selected_features)
    df = df.set_index('time')
    combined_df = df.merge(features, left_index=True, right_index=True, how='left')

    missing_features = [feat for feat in selected_features if feat not in combined_df.columns]
    for feat in missing_features:
        combined_df[feat] = 0

    combined_df = combined_df[selected_features]

    combined_df = combined_df.dropna()

    return combined_df, mean_candle_size

# Calculate SL and TP based on entry price and specified percentages
# Calculate SL and TP based on entry price and specified percentages, considering Buy and Sell separately
def calculate_prices(entry_price, risk_reward_ratio, mean_candle_size, trade_type):
    risk_part, reward_part = map(int, risk_reward_ratio.split(':'))
    risk_amount = mean_candle_size * risk_part
    reward_amount = mean_candle_size * reward_part

    if trade_type == "Buy":
        sl_price = entry_price - risk_amount  # For Buy, SL is below the entry price
        tp_price = entry_price + reward_amount  # For Buy, TP is above the entry price
    elif trade_type == "Sell":
        sl_price = entry_price + risk_amount  # For Sell, SL is above the entry price
        tp_price = entry_price - reward_amount  # For Sell, TP is below the entry price
    else:
        raise ValueError(f"Invalid trade_type: {trade_type}. It must be 'Buy' or 'Sell'.")
    
    return sl_price, tp_price


# Place order with retry mechanism
def place_order(symbol, volume, sl_price, tp_price, trade_type, config, retry_attempts=3):
    for attempt in range(retry_attempts):
        with mt5_lock:
            price = mt5.symbol_info_tick(symbol).ask if trade_type == "Buy" else mt5.symbol_info_tick(symbol).bid
            order_type = mt5.ORDER_TYPE_BUY if trade_type == "Buy" else mt5.ORDER_TYPE_SELL
            request = {
                "action": mt5.TRADE_ACTION_DEAL,
                "symbol": symbol,
                "volume": volume,
                "type": order_type,
                "price": price,
                "sl": sl_price,
                "tp": tp_price,
                "deviation": 10,
                "magic": config["magic"],
                "comment": f"Python script {trade_type} order",
                "type_time": mt5.ORDER_TIME_GTC,
                "type_filling": mt5.ORDER_FILLING_IOC,
            }

            result = mt5.order_send(request)
            if result.retcode == mt5.TRADE_RETCODE_DONE:
                logging.info(f"{trade_type} order placed successfully for {symbol}.")
                print(f"{trade_type} order placed successfully for {symbol}.")
                return True
            else:
                logging.error(f"Order placement failed for {symbol}: {result.retcode}. Retrying...")
                print(f"Order placement failed for {symbol}: {result.retcode}. Retrying...")

        time.sleep(2)  # Sleep briefly before retrying

    logging.error(f"Order failed after {retry_attempts} attempts for {symbol}")
    return False

# Predict and gather both Buy and Sell predictions first before placing any trade
def predict_and_gather_predictions(scaler, model, symbol, timeframe, start_date, end_date, features, trade_type, threshold=None):
    signals = ['WILLR_15', 'WILLR_42']  # Signals used for both buy and sell

    # Fetch and process data
    df_processed, mean_candle_size = fetch_and_process_data(symbol, timeframe, signals, features, start_date, end_date)

    if df_processed is None:
        logging.error(f"No valid data for {symbol}. Skipping predictions.")
        return None, mean_candle_size

    # Shift the data
    df_processed = df_processed.shift(periods=1, axis=0).dropna()

    # Align the features with what the scaler expects
    df_processed = df_processed[features]

    # Scale the data
    scaled_data = scaler.transform(df_processed)

    # Predict probabilities and apply the provided threshold or default to 0.5
    probas = model.predict_proba(scaled_data)[:, 1]
    custom_threshold = threshold if threshold is not None else 0.5
    y_pred = (probas >= custom_threshold).astype(int)   
    pred = y_pred[-1] if len(y_pred) > 0 else 0
    return pred, mean_candle_size  # Return last prediction and mean candle size 

# Decision function to check both Buy and Sell predictions
# Decision function to check both Buy and Sell predictions
def decide_and_trade(buy_pred, sell_pred, symbol, volume, risk_reward_ratio, mean_candle_size, config):
    if buy_pred == 1 and sell_pred == 1:
        logging.info(f"Both Buy and Sell predictions are 1 for {symbol}. No trade will be taken.")
        print(f"Both Buy and Sell predictions are 1 for {symbol}. No trade will be taken.")
        return

    if buy_pred == 1 and sell_pred == 0:
        entry_price = mt5.symbol_info_tick(symbol).ask
        sl_price, tp_price = calculate_prices(entry_price, risk_reward_ratio, mean_candle_size, "Buy")
        place_order(symbol, volume, sl_price, tp_price, "Buy", config)

    if sell_pred == 1 and buy_pred == 0:
        entry_price = mt5.symbol_info_tick(symbol).bid
        sl_price, tp_price = calculate_prices(entry_price, risk_reward_ratio, mean_candle_size, "Sell")
        place_order(symbol, volume, sl_price, tp_price, "Sell", config)


# Refactored function to predict both buy and sell and then decide
def process_pair(config, utc_from, utc_to, trade_decisions):
    logging.info(f"Processing pair: {config['symbol']} for Buy and Sell")

    buyscaler, buymodel, sellscaler, sellmodel = None, None, None, None

    try:
        buyscaler = load(config["buy_scaler_path"])
        buymodel = load(config["buy_model_path"])

        with open(config["buy_features_path"], 'r') as f:
            buy_features = json.load(f)
    except Exception as e:
        logging.warning(f"Buy model or scaler missing for {config['symbol']}: {e}")
        print(f"Buy model or scaler missing for {config['symbol']}: {e}")

    try:
        if "sell_scaler_path" in config and "sell_model_path" in config:
            sellscaler = load(config["sell_scaler_path"])
            sellmodel = load(config["sell_model_path"])

            with open(config["sell_features_path"], 'r') as f:
                sell_features = json.load(f)
        else:
            logging.warning(f"Sell model or scaler missing for {config['symbol']}")
            print(f"Sell model or scaler missing for {config['symbol']}")
    except Exception as e:
        logging.warning(f"Error loading sell model for {config['symbol']}: {e}")
        print(f"Error loading sell model for {config['symbol']}: {e}")

    try:
        with mt5_lock:
            init_mt5_connection(config['login'], config['password'], config['server'], config['terminal_path'])

        while True:
            current_time = datetime.now(pytz.utc)

            if current_time.hour == 22 and current_time.minute == 30:  # Example trading window
                buy_pred, mean_candle_size = None, None
                sell_pred = None

                if buyscaler is not None and buymodel is not None:
                    buy_pred, mean_candle_size = predict_and_gather_predictions(
                        buyscaler, buymodel, config["symbol"], config["timeframe"], utc_from, utc_to, buy_features, "Buy",
                        threshold=config.get("buy_threshold", None)
                    )

                if sellscaler is not None and sellmodel is not None:
                    sell_pred, _ = predict_and_gather_predictions(
                        sellscaler, sellmodel, config["symbol"], config["timeframe"], utc_from, utc_to, sell_features, "Sell",
                        threshold=config.get("sell_threshold", None)
                    )

                if buy_pred is not None and sell_pred is not None:
                    decide_and_trade(buy_pred, sell_pred, config["symbol"], config["volume"], config["buy_risk_reward_ratio"], mean_candle_size, config)

                logging.info(f"Finished trading for {config['symbol']} at 00:20 UTC")

            time.sleep(60)  # Sleep for a minute and recheck
    except KeyboardInterrupt:
        logging.info(f"Script terminated by user for {config['symbol']}.")
        print(f"Script terminated by user for {config['symbol']}.")
    finally:
        mt5.shutdown()
        logging.info(f"MetaTrader 5 connection closed for {config['symbol']}.")
        print(f"MetaTrader 5 connection closed for {config['symbol']}.")

# Main function
def main():
    kill_mt5_instances()

    eurusd_config = {
        "symbol": "EURUSD",
        "login": 51988090,
        "password": '1fMdV52$74EOcw',
        "server": 'ICMarketsEU-Demo',
        "terminal_path": r"C:\Program Files\MetaTrader5ICMarketsEU2\terminal64.exe",
        "timeframe": mt5.TIMEFRAME_D1,
        "volume": 0.1,
        "buy_risk_reward_ratio": "2:3",
        "sell_risk_reward_ratio": "2:3",
        "buy_scaler_path": 'EURUSD_D1_3112buy/scaler.joblib',
        "buy_model_path": 'EURUSD_D1_3112buy/model.joblib',
        "sell_scaler_path": 'EURUSD_D1_3112sell/scaler.joblib',
        "sell_model_path": 'EURUSD_D1_3112sell/model.joblib',
        "buy_features_path": 'EURUSD_D1_3112buy/feature_names.json',
        "sell_features_path": 'EURUSD_D1_3112sell/feature_names.json',
        "magic": 1001,
    }

    gbpusd_config = {
        "symbol": "GBPUSD",
        "login": 51988092,
        "password": 'ty!H!03FNx!kEh',
        "server": 'ICMarketsEU-Demo',
        "terminal_path": r"C:\Program Files\MetaTrader5ICMarketsEU3\terminal64.exe",
        "timeframe": mt5.TIMEFRAME_D1,
        "volume": 0.1,
        "buy_risk_reward_ratio": "1:1",
        "sell_risk_reward_ratio": "1:1",
        "buy_scaler_path": 'GBPUSD_D1_3112buy/scaler.joblib',
        "buy_model_path": 'GBPUSD_D1_3112buy/model.joblib',
        "sell_scaler_path": 'GBPUSD_D1_3112sell/scaler.joblib',
        "sell_model_path": 'GBPUSD_D1_3112sell/model.joblib',
        "buy_features_path": 'GBPUSD_D1_3112buy/feature_names.json',
        "sell_features_path": 'GBPUSD_D1_3112sell/feature_names.json',
        "magic": 1002,
    }

    usdcad_config = {
        "symbol": "USDCAD",
        "login": 51988094,
        "password": '!rwj8zZ5iA$Sz5',
        "server": 'ICMarketsEU-Demo',
        "terminal_path": r"C:\Program Files\MetaTrader5ICMarketsEU4\terminal64.exe",
        "timeframe": mt5.TIMEFRAME_D1,
        "volume": 0.1,
        "buy_risk_reward_ratio": "2:3",
        "sell_risk_reward_ratio": "2:3",
        "buy_scaler_path": 'USDCAD_D1_3112buy/scaler.joblib',
        "buy_model_path": 'USDCAD_D1_3112buy/model.joblib',
        "sell_scaler_path": 'USDCAD_D1_3112_Sell/scaler.joblib',
        "sell_model_path": 'USDCAD_D1_3112_Sell/model.joblib',
        "buy_features_path": 'USDCAD_D1_3112buy/feature_names.json',
        "sell_features_path": 'USDCAD_D1_3112_Sell/feature_names.json',
        "magic": 1003,
    }

    configs = [eurusd_config, gbpusd_config, usdcad_config]

    utc_from = datetime(2023, 1, 1, tzinfo=pytz.utc)
    utc_to = datetime.now(pytz.utc)

    trade_decisions = {config['symbol']: {'buy': 0, 'sell': 0} for config in configs}

    threads = []
    for config in configs:
        thread = threading.Thread(target=process_pair, args=(config, utc_from, utc_to, trade_decisions))
        thread.start()
        threads.append(thread)

    for thread in threads:
        thread.join()

if __name__ == "__main__":
    main()

c:\ProgramData\anaconda3\envs\forex_env\Lib\site-packages\dask\dataframe\_pyarrow_compat.py:23: UserWarning: You are using pyarrow version 11.0.0 which is known to be insecure. See https://www.cve.org/CVERecord?id=CVE-2023-47248 for further details. Please upgrade to pyarrow>=14.0.1 or install pyarrow-hotfix to patch your current version.
  warnings.warn(


Error loading sell model for EURUSD: [Errno 2] No such file or directory: 'EURUSD_D1_3112sell/scaler.joblib'
Connected to MetaTrader 5 with login: 51988090
Connected to MetaTrader 5 with login: 51988094
Connected to MetaTrader 5 with login: 51988092
